In [ ]:
import serial
import time

class ColdPlateSlimDriver:
    def __init__(
            self,
            protocol_context,
            temp_mode_number,
            max_temp_lag=0,
            heating_rate_deg_per_min=100,
            cooling_rate_deg_per_min=100,
    ):
                # Ensure temp_mode_number is an integer
        if not isinstance(temp_mode_number, int):
            raise ValueError(f"temp_mode_number must be an integer, not {type(temp_mode_number)}")
        
        self.serial_number = "29517"
        # Directly convert temp_mode_number to a string here
        self.device_name = "/dev/ttyUSB" + str(temp_mode_number)
        self.baudrate = 9600
        self.bytesize = serial.EIGHTBITS
        self.parity = serial.PARITY_NONE
        self.stopbits = serial.STOPBITS_ONE
        self.read_timeout = 2
        self.write_timeout = 2
        self.height = 45
        self.temp = 0
        self.deck_position = 7  # Replace this with labwarePosition_heat_module or w.e
        self.max_temp_lag = max_temp_lag
        self.heating_rate_deg_per_min = heating_rate_deg_per_min
        self.cooling_rate_deg_per_min = cooling_rate_deg_per_min
        self.protocol = protocol_context

        # check context, skip if simulating Linux
        if protocol_context.is_simulating():
            print("simulation detected")
            print("Initializing in the dummy mode")
            self.serial_object = None
            self.max_temp_lag = 0
            self.heating_rate_deg_per_min = 1000
            self.cooling_rate_deg_per_min = 1000
        else:
            print("execution mode")
            print("Initializing in the real deal mode")
            self.serial_object = serial.Serial(
                port=self.device_name,
                baudrate=self.baudrate,
                bytesize=self.bytesize,
                parity=self.parity,
                stopbits=self.stopbits,
                timeout=self.read_timeout,
                write_timeout=self.write_timeout)

    def _reset_buffers(self):
        """
        Worker function
        """
        if self.serial_object is None:
            return
        self.serial_object.reset_input_buffer()
        self.serial_object.reset_output_buffer()

    def _read_response(self):
        """
        Worker function
        """
        if self.serial_object is None:
            return ("dummy response")

        output_lines = self.serial_object.readlines()
        output_string = "".join(l.decode("utf-8") for l in output_lines)
        return output_string

    def _send_command(self, my_command):
        """
        Worker function
        """
        SERIAL_ACK = "\r\n"

        command = my_command
        command += SERIAL_ACK

        if self.serial_object is None:
            print("sending dummy command: " + my_command)
            return

        self.serial_object.write(command.encode())
        self.serial_object.flush()
        return (self._read_response())

    def get_info(self):
        if self.serial_object is None:
            return "dummy info"
        return self._send_command("info")

    def get_temp(self):
        if self.serial_object is None:
            return self.temp
        t = self._send_command("getTempActual")
        return float(t)

    def set_temp(self, my_temp):
        if self.serial_object is None:
            self.temp = my_temp
            return

        temp = float(my_temp) * 10
        temp = int(temp)
        self._send_command(f'setTempTarget{temp:03}')
        self._send_command("tempOn")
        return

    def set_temp_andWait(self, target_temp, timeout_min=30, tolerance=0.5):

        interval_sec = 10
        SEC_IN_MIN = 60

        curr_temp = self.get_temp()
        self.protocol.comment(f"Setting temperature. Current temp: {curr_temp}\nTarget temp: {target_temp}")

        temp_diff = target_temp - curr_temp
        temp_lag = self.max_temp_lag * (abs(temp_diff) / 100.0)

        if temp_diff > 0:
            temp_step = self.heating_rate_deg_per_min * (interval_sec / SEC_IN_MIN)
            self.protocol.comment(f"Heating rate: {temp_step}")
        else:
            temp_step = -self.cooling_rate_deg_per_min * (interval_sec / SEC_IN_MIN)
            self.protocol.comment(f"Cooling rate: {temp_step}")

        while (abs(target_temp - curr_temp) > abs(temp_step)):
            curr_temp += temp_step
            self.set_temp(curr_temp)
            self.protocol.comment(f"Ramping the temp to: {curr_temp}")
            self.protocol.delay(seconds=interval_sec)
            curr_temp = self.get_temp()
            self.protocol.comment(f"Actual temp: {curr_temp}")

        self.set_temp(target_temp)

        time_elapsed = 0

        while (abs(self.get_temp() - target_temp) > tolerance):
            self.protocol.comment(f"Waiting for temp to reach target: {target_temp}, actual temp: {self.get_temp()}")
            if not self.protocol.is_simulating():  # Skip delay during simulation
                self.protocol.delay(seconds=interval_sec)
            time_elapsed += interval_sec
            if (time_elapsed > timeout_min * 60):
                raise Exception("Temperature timeout")

        self.protocol.comment(f"Target reached, equilibrating for {ab_incubation_time_minutes} minutes")
        if not self.protocol.is_simulating():  # Skip delay during simulation
            self.protocol.delay(seconds=temp_lag * SEC_IN_MIN)
        return target_temp

    def temp_off(self):
        if self.serial_object is None:
            self.temp = 25
        else:
            self._send_command("tempOff")
    def quick_temp(self, temp_target, overshot=10):

        start_temp = self.get_temp()
        end_time = 120 * 60  # 2 hours in seconds

        # Calculate the overshot temperature based on the direction of the temperature change
        if temp_target > start_temp:
            overshot_temp = temp_target + overshot
        else:
            overshot_temp = temp_target - overshot

        # Set the temperature to the overshot temperature first
        self.set_temp(overshot_temp)

        # Initialize the elapsed time
        elapsed = 0
        interval = 10  # Check every 10 seconds

        # Monitoring loop with auto-shutoff after 2 hours
        while elapsed < end_time:
            current_temp = self.get_temp()
            print(f"time elapsed (min): \t{elapsed / 60:.1f}")
            print(f"temp_mod       (C): \t{current_temp}")

            # Check if we've reached or passed the target temperature
            if (temp_target > start_temp and current_temp >= temp_target) or \
               (temp_target < start_temp and current_temp <= temp_target):
                break

            time.sleep(interval)
            elapsed += interval

        # After overshooting, set to the target temperature
        self.set_temp(temp_target)

        # Continue monitoring until 2 hours have elapsed
        while elapsed < end_time:
            current_temp = self.get_temp()
            print(f"time elapsed (min): \t{elapsed / 60:.1f}")
            print(f"temp_mod       (C): \t{current_temp}")
            time.sleep(interval)
            elapsed += interval

        # Auto-shutoff after 2 hours
        if elapsed >= end_time:
            self.temp_off()
            print("Auto-shutoff triggered after 2 hours.")
            
    def quick_temp2(self, temp_target, shutoff_time_minutes=120, overshot=10):


        start_temp = self.get_temp()
        end_time = shutoff_time_minutes * 60  # Convert minutes to seconds

        # Calculate the overshot temperature based on the direction of the temperature change
        if temp_target > start_temp:
            overshot_temp = temp_target + overshot
        else:
            overshot_temp = temp_target - overshot

        # Set the temperature to the overshot temperature first
        self.set_temp(overshot_temp)

        # Initialize the elapsed time
        elapsed = 0
        interval = 10  # Check every 10 seconds

        # Monitoring loop with auto-shutoff based on the provided time
        while elapsed < end_time:
            current_temp = self.get_temp()
            print(f"time elapsed (min): \t{elapsed / 60:.1f}")
            print(f"temp_mod       (C): \t{current_temp}")

            # Check if we've reached or passed the target temperature
            if (temp_target > start_temp and current_temp >= temp_target) or \
               (temp_target < start_temp and current_temp <= temp_target):
                break

            time.sleep(interval)
            elapsed += interval

        # After overshooting, set to the target temperature
        self.set_temp(temp_target)

        # Continue monitoring until the specified shutoff time has elapsed
        while elapsed < end_time:
            current_temp = self.get_temp()
            print(f"time elapsed (min): \t{elapsed / 60:.1f}")
            print(f"temp_mod       (C): \t{current_temp}")
            time.sleep(interval)
            elapsed += interval

        # Auto-shutoff after the specified time
        if elapsed >= end_time:
            self.temp_off()
            print(f"Auto-shutoff triggered after {shutoff_time_minutes} minutes.")




In [ ]:
import sys

sys.path.append('/var/lib/jupyter/notebooks')

from opentrons import protocol_api

import opentrons.execute
import opentrons.simulate

# import opentrons.simulate if you want to simulate the protocol first

# This is where you establish the API version for executing a protocol
protocol = opentrons.execute.get_protocol_api('2.13')


In [1]:
#the temp mod number (zero below) may need to change
temp_mod = ColdPlateSlimDriver(protocol,0)


NameError: name 'ColdPlateSlimDriver' is not defined

In [ ]:
temp_mod.set_temp(90.0)

In [ ]:
temp_mod.quick_temp2(10.0, 5)
temp_mod.quick_temp2(90.0, 50)

In [ ]:
temp_mod._send_command("tempOff")
print(f"Turning off")

In [ ]:
#WARNING ONLY INITIATE THIS CELL IF RECALIBRATING OVERCLOCK
t1 = temp_mod._send_command("getTemp1ActualValue")
print(f"{t1} T1")

temp_mod._send_command("setTemp1ActualValue072")


t1 = temp_mod._send_command("getTemp1ActualValue")
print(f"{t1} T1")

t2 = temp_mod._send_command("getTemp2ActualValue")
print(f"{t2} T2")

temp_mod._send_command("setTemp2ActualValue172")


t2 = temp_mod._send_command("getTemp2ActualValue")
print(f"{t2} T2")

t3 = temp_mod._send_command("getTemp3ActualValue")
print(f"{t3} T3")
temp_mod._send_command("setTemp3ActualValue396")


t3 = temp_mod._send_command("getTemp3ActualValue")
print(f"{t3} T3")

t4 = temp_mod._send_command("getTemp4ActualValue")
print(f"{t4} T4")
temp_mod._send_command("setTemp4ActualValue843")


t4 = temp_mod._send_command("getTemp4ActualValue")
print(f"{t4} T4")

In [ ]:
##WARNING THIS REMOVES OVERCLOCK
#if an error occurs, do this
temp_mod._send_command("resetDevice")

In [ ]:
#WARNING ONLY INITIATE THIS CELL IF RECALIBRATING OVERCLOCK
t1 = temp_mod._send_command("getTemp1ActualValue")
print(f"{t1} T1")

temp_mod._send_command("setTemp1ActualValue072")


t1 = temp_mod._send_command("getTemp1ActualValue")
print(f"{t1} T1")

t2 = temp_mod._send_command("getTemp2ActualValue")
print(f"{t2} T2")

temp_mod._send_command("setTemp2ActualValue172")


t2 = temp_mod._send_command("getTemp2ActualValue")
print(f"{t2} T2")

t3 = temp_mod._send_command("getTemp3ActualValue")
print(f"{t3} T3")
temp_mod._send_command("setTemp3ActualValue396")


t3 = temp_mod._send_command("getTemp3ActualValue")
print(f"{t3} T3")

t4 = temp_mod._send_command("getTemp4ActualValue")
print(f"{t4} T4")
temp_mod._send_command("setTemp4ActualValue843")


t4 = temp_mod._send_command("getTemp4ActualValue")
print(f"{t4} T4")